<a href="https://colab.research.google.com/github/cavadmanafov/nlp-intro/blob/main/introduction_to_npl_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-03-13 05:35:30--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-13 05:35:31 (84.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-03-13 05:35:36--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.198.128, 64.233.191.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.198.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-03-13 05:35:36 (60.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data("nlp_getting_started.zip")

In [ ]:
import pandas as pd 

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
import random
random_index = random.randint(0, len(train_df) - 5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index + 5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")


Target: 0 (not real disaster)
Text:
@KamKasteIIano @BluntedJayt FUCKING SCREAMING !

---

Target: 0 (not real disaster)
Text:
ColnHarun: Breaking news! Unconfirmed! I just heard a loud bang nearby. in what appears to be a blast of wind from my neighbour's ass.

---

Target: 1 (real disaster)
Text:
@OllyMursAus I do feel sorry for him! He is not a piece of meat! He is a nice guy... People don't need to rush him and screams in his face!

---

Target: 0 (not real disaster)
Text:
@Mmchale13 *tries to electrocute self with phone cord*

---

Target: 1 (real disaster)
Text:
@SatanaOfHell ever seen by far. A dreamy look came over his face and his ears flattened as he started to walk slowly toward her.* You'd-

---



In [ ]:
 from sklearn.model_selection import train_test_split

In [ ]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
text_vectorizer = TextVectorization(max_tokens = 10000,
                                 output_mode = "int",
                                 output_sequence_length = 15
                                )

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
random_sentece = random.choice(train_sentences)

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [ ]:
max_vocab_length = 10000
max_length = 15


In [ ]:
embedding = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                          output_dim = 128,
                          input_length = max_length)

embedding

In [ ]:
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 WSJThinkTank: Ahead of tonight's #GOPDebate ColleenMNelson explains how a bad debate can derail a campaign: Û_ http://t.co/XyxTuACZvb      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02031419,  0.03753212,  0.00782609, ...,  0.00236405,
          0.01990335, -0.03744709],
        [ 0.0370496 , -0.04330472,  0.00401543, ..., -0.01325787,
          0.04573962, -0.01792326],
        [-0.01666128, -0.00978441, -0.03843856, ...,  0.04761273,
          0.01161374,  0.02380544],
        ...,
        [-0.01598302,  0.04826354,  0.03723133, ...,  0.01363591,
         -0.04652143, -0.0319697 ],
        [-0.02511276,  0.03580299, -0.02514938, ...,  0.02699523,
          0.04679156,  0.04402424],
        [-0.02080506,  0.03864172, -0.00030822, ...,  0.0292244 ,
          0.02216801, -0.04770095]]], dtype=float32)>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
 baseline_score = model_0.score(val_sentences, val_labels)
 print(f"Our accuracy {baseline_score * 100:.2f}%")

Our accuracy 79.27%


In [ ]:
baseline_preds = model_0.predict(val_sentences)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")
  model_results = {"accuracy": model_accuracy,
                   "recall": model_recall,
                   "precision": model_precision,
                   "f1": model_f1}
  return model_results


In [ ]:
baseline_results = calculate_results(y_true = val_labels, y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'recall': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'f1': 0.7862189758049549}

In [ ]:
train_labels[:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [ ]:
inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string, name = "input_layer")
x = text_vectorizer(inputs) # turn the inputs text to numbers
x = embedding(x) # create an embedding of the numberized inputs
x = tf.keras.layers.GlobalAveragePooling1D(name = "gloval_average_pooling1d_layer")(x)
outputs = tf.keras.layers.Dense(1, activation = "sigmoid", name = "output_layer")(x)
model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [ ]:
model_1.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model_1_history = model_1.fit(x = train_sentences,
                              y=  train_labels,
                              validation_data = (val_sentences, val_labels),
                              epochs = 5,
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230313-053546
Epoch 1/5
215/215 [==============================] - 28s 88ms/step - loss: 0.6106 - accuracy: 0.6878 - val_loss: 0.5364 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4419 - accuracy: 0.8193 - val_loss: 0.4764 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3478 - accuracy: 0.8606 - val_loss: 0.4580 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2844 - accuracy: 0.8926 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2386 - accuracy: 0.9123 - val_loss: 0.4827 - val_accuracy: 0.7808


In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [ ]:
model_1_probs = tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]  

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create a LSTM model

inputs = tf.keras.layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.LSTM(64, return_sequences = True)(x)
x = tf.keras.layers.LSTM(64)(x)
# x = tf.keras.layers.Dense(64, activation = "relu")(x)
outputs = tf.keras.layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")




In [ ]:
model_2.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                             experiment_name = "model_2_LSTM")] 
)

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230313-053624
Epoch 1/5
215/215 [==============================] - 13s 43ms/step - loss: 0.2214 - accuracy: 0.9259 - val_loss: 0.5485 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1565 - accuracy: 0.9448 - val_loss: 0.6062 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1269 - accuracy: 0.9533 - val_loss: 0.7034 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1042 - accuracy: 0.9618 - val_loss: 0.9312 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0832 - accuracy: 0.9680 - val_loss: 0.9544 - val_accuracy: 0.7808


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[5.2827317e-03],
       [7.5780052e-01],
       [9.9972588e-01],
       [5.5328459e-02],
       [6.7040225e-04],
       [9.9880934e-01],
       [8.9807242e-01],
       [9.9980170e-01],
       [9.9973077e-01],
       [5.1647204e-01]], dtype=float32)

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string, name = "input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GRU(64, return_sequences = True)(x)
x = tf.keras.layers.GlobalAveragePooling1D(name = "global_average_pooling_layer")(x)
outputs = tf.keras.layers.Dense(1, activation = "sigmoid", name = "output_layer")(x)
model_3 = tf.keras.Model(inputs, outputs, name = "model_3_GRU")


In [ ]:
model_3.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              validation_data = (val_sentences, val_labels),
                              epochs = 5,
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230313-053708
Epoch 1/5
215/215 [==============================] - 11s 39ms/step - loss: 0.1751 - accuracy: 0.9467 - val_loss: 0.7224 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1020 - accuracy: 0.9628 - val_loss: 0.8247 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0860 - accuracy: 0.9673 - val_loss: 0.8842 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0750 - accuracy: 0.9705 - val_loss: 1.1108 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0640 - accuracy: 0.9746 - val_loss: 1.1906 - val_accuracy: 0.7664


In [ ]:
inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string, name = "input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.Bidertional(tf.keras.layers.LSTM(64, return_sequences = True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs)


In [ ]:
model_4.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              validation_data = (val_sentences, val_labels),
                              epochs = 5,
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "model_4_birdirectional")])

Saving TensorBoard log files to: model_logs/model_4_birdirectional/20230313-053731
Epoch 1/5
215/215 [==============================] - 14s 43ms/step - loss: 0.1123 - accuracy: 0.9688 - val_loss: 0.8527 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0560 - accuracy: 0.9768 - val_loss: 1.0290 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0499 - accuracy: 0.9775 - val_loss: 1.1050 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0489 - accuracy: 0.9783 - val_loss: 1.1759 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0426 - accuracy: 0.9797 - val_loss: 1.4552 - val_accuracy: 0.7598


In [ ]:
inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string, name = "input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(filters = 64, kernel_size = 5, strides = 1, activation = "relu", padding = "valid")(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1, activation = "sigmoid", name = "output_layer")(x)
model_5 = tf.keras.Model(inputs, outputs)

model_5.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model_5.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 output_layer (Dense)        (None, 1)                 65  

In [ ]:
model_5_history = model_5.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                             experiment_name = "model_5_conv1d")]
)

Saving TensorBoard log files to: model_logs/model_5_conv1d/20230313-053756
Epoch 1/5
215/215 [==============================] - 16s 46ms/step - loss: 0.1200 - accuracy: 0.9588 - val_loss: 0.9183 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0706 - accuracy: 0.9730 - val_loss: 1.0562 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0609 - accuracy: 0.9766 - val_loss: 1.1493 - val_accuracy: 0.7507
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0530 - accuracy: 0.9769 - val_loss: 1.2328 - val_accuracy: 0.7480
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0493 - accuracy: 0.9788 - val_loss: 1.3114 - val_accuracy: 0.7480


In [ ]:
import tensorflow_hub as hub
sample_sentence = "Theres is a flood in my street"
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embed_samples = embed([sample_sentence,
                       "When you can the universal sample sentence, it turns to good thing. "])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01999853  0.01877251  0.02695384  0.00712348  0.02266598  0.08827806
  0.02436963  0.06032907  0.00068981 -0.00044289 -0.00666334 -0.017351
 -0.00222248  0.10334865  0.05820671 -0.03342599  0.01705527 -0.04488812
  0.02095671 -0.07260092 -0.05648819  0.02149009  0.03515625 -0.00527486
  0.00580678 -0.04299988  0.01399155 -0.00857992 -0.03540717 -0.0092045
 -0.04168848  0.05664096 -0.02471749  0.01061325  0.0235052  -0.07981201
  0.02338948  0.04581924 -0.01548058 -0.08292788  0.02199061 -0.01058304
 -0.01481375  0.09282359 -0.10695367 -0.0634724   0.01233517 -0.02458699
 -0.02198262  0.03864015], shape=(50,), dtype=float32)


In [ ]:
sentence_encoding_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                         input_shape = [],
                                         trainable = False,
                                         name = "USE",
                                         dtype = tf.string)


In [ ]:
model_6 = tf.keras.Sequential([
    sentence_encoding_layer,
    tf.keras.layers.Dense(64, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [ ]:
model_6.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,
                                                 val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "model_6_hub")])

Saving TensorBoard log files to: model_logs/model_6_hub/20230313-054003
Epoch 1/5
215/215 [==============================] - 10s 27ms/step - loss: 0.5030 - accuracy: 0.7878 - val_loss: 0.4487 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4145 - accuracy: 0.8177 - val_loss: 0.4388 - val_accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4001 - accuracy: 0.8247 - val_loss: 0.4370 - val_accuracy: 0.8071
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3921 - accuracy: 0.8267 - val_loss: 0.4279 - val_accuracy: 0.8097
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3856 - accuracy: 0.8298 - val_loss: 0.4215 - val_accuracy: 0.8202


In [ ]:
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [ ]:
#train_10_percent = train_df_shuffled[["text", "target"]].sample##(frac = 0.1, random_state = 42)
#train_sentences_10_percent = train_10_percent["text"].to_list()
#train_labels_10_percent = train_10_percent["target"].to_list()


In [ ]:
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
len(train_labels_10_percent)
len(train_sentences_10_percent)

685

In [ ]:
model_7 = tf.keras.models.clone_model(model_6)

In [ ]:
model_7.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model_7_history = model_7.fit(
    train_sentences_10_percent,
    train_labels_10_percent,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                 experiment_name = "model_7_10_percent")])

Saving TensorBoard log files to: model_logs/model_7_10_percent/20230313-065718
Epoch 1/5
22/22 [==============================] - 5s 81ms/step - loss: 0.6644 - accuracy: 0.6832 - val_loss: 0.6453 - val_accuracy: 0.7113
Epoch 2/5
22/22 [==============================] - 1s 22ms/step - loss: 0.5931 - accuracy: 0.7956 - val_loss: 0.5866 - val_accuracy: 0.7520
Epoch 3/5
22/22 [==============================] - 0s 22ms/step - loss: 0.5164 - accuracy: 0.8248 - val_loss: 0.5374 - val_accuracy: 0.7743
Epoch 4/5
22/22 [==============================] - 1s 23ms/step - loss: 0.4588 - accuracy: 0.8248 - val_loss: 0.5049 - val_accuracy: 0.7769
Epoch 5/5
22/22 [==============================] - 0s 21ms/step - loss: 0.4190 - accuracy: 0.8365 - val_loss: 0.4926 - val_accuracy: 0.7769
